In [1]:
import torch
from diffusers import SanaPipeline

pipe = SanaPipeline.from_pretrained(
    "Efficient-Large-Model/Sana_1600M_1024px_diffusers",
    #"Efficient-Large-Model/Sana_600M_1024px_diffusers",
    variant="fp16",
    torch_dtype=torch.float16,
)
pipe.to("cuda")

pipe.vae.to(torch.bfloat16)
pipe.text_encoder.to(torch.bfloat16)


/home/galkesten/miniconda3/envs/sana/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/galkesten/miniconda3/envs/sana/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "/home/galkesten/miniconda3/envs/sana/lib/python3.10/site-packages/xformers/__init__.py", line 57, in _is_triton_available
    import triton  # noqa
  File "/home/galkesten/minicond

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Gemma2Model(
  (embed_tokens): Embedding(256000, 2304, padding_idx=0)
  (layers): ModuleList(
    (0-25): 26 x Gemma2DecoderLayer(
      (self_attn): Gemma2Attention(
        (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
        (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
        (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
        (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        (rotary_emb): Gemma2RotaryEmbedding()
      )
      (mlp): Gemma2MLP(
        (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
        (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
        (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
        (act_fn): PytorchGELUTanh()
      )
      (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
      (pre_feedforward_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
      (post_feedforward_layernorm): Gemma2RMSNorm((2304,),

In [2]:
pipe

SanaPipeline {
  "_class_name": "SanaPipeline",
  "_diffusers_version": "0.34.0.dev0",
  "_name_or_path": "Efficient-Large-Model/Sana_1600M_1024px_diffusers",
  "scheduler": [
    "diffusers",
    "DPMSolverMultistepScheduler"
  ],
  "text_encoder": [
    "transformers",
    "Gemma2Model"
  ],
  "tokenizer": [
    "transformers",
    "GemmaTokenizerFast"
  ],
  "transformer": [
    "diffusers",
    "SanaTransformer2DModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderDC"
  ]
}

In [3]:
len(pipe.transformer.transformer_blocks)

20

In [4]:
pipe.enable_attention_slicing()

In [5]:
import torch
import os
import time

def zero_ablation_hook(module, input, output):
    return torch.zeros_like(output)

def mean_ablation_hook(module, input, output):
    return output.mean(dim=1, keepdim=True).expand_as(output)

# === Create output directory ===
os.makedirs("ablated_outputs", exist_ok=True)

# === Define prompt ===
prompt = "a photo of a cow right of a laptop"

# === Generate and save baseline (no ablation) ===
print("Generating baseline image (no ablation)")
baseline_image = pipe(
    prompt=prompt,
    height=1024,
    width=1024,
    guidance_scale=5.0,
    num_inference_steps=20,
    generator=torch.Generator(device="cuda").manual_seed(42),
)[0]
baseline_image[0].save("ablated_outputs/baseline_no_ablation.png")

# === Iterate over transformer blocks for zero ablation ===
for idx, block in enumerate(pipe.transformer.transformer_blocks):
    print(f"Zero ablating FFN in block {idx}")

    # Register hook
    handle = block.ff.register_forward_hook(zero_ablation_hook)

    # Measure time
    start_time = time.time()
    image = pipe(
        prompt=prompt,
        height=1024,
        width=1024,
        guidance_scale=5.0,
        num_inference_steps=20,
        generator=torch.Generator(device="cuda").manual_seed(42),
    )[0]
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Generation time for block {idx}: {elapsed_time:.2f} seconds")

    # Save image
    image[0].save(f"ablated_outputs/ablated_block{idx}_ffn_zero.png")

    # Remove hook
    handle.remove()

# # === Optional: Repeat for mean ablation ===
# for idx, block in enumerate(pipe.transformer.transformer_blocks):
#     print(f"Mean ablating FFN in block {idx}")

#     # Register hook
#     handle = block.ff.register_forward_hook(mean_ablation_hook)

#     # Generate image
#     image = pipe(
#         prompt=prompt,
#         height=1024,
#         width=1024,
#         guidance_scale=5.0,
#         num_inference_steps=20,
#         generator=torch.Generator(device="cuda").manual_seed(42),
#     )[0]

#     # Save image
#     image[0].save(f"ablated_outputs/ablated_block{idx}_ffn_mean.png")

#     # Remove hook
#     handle.remove()


Generating baseline image (no ablation)


  0%|          | 0/20 [00:00<?, ?it/s]

Zero ablating FFN in block 0


  0%|          | 0/20 [00:00<?, ?it/s]

Generation time for block 0: 2.99 seconds
Zero ablating FFN in block 1


...

Generation time for block 17: 3.01 seconds
Zero ablating FFN in block 18


  0%|          | 0/20 [00:00<?, ?it/s]

Generation time for block 18: 3.01 seconds
Zero ablating FFN in block 19


  0%|          | 0/20 [00:00<?, ?it/s]

Generation time for block 19: 3.01 seconds


In [6]:
import torch
import os

def zero_ablation_hook(module, input, output):
    return torch.zeros_like(output)

def mean_ablation_hook(module, input, output):
    ret= output.mean(dim=1, keepdim=True).expand_as(output)
    print(ret.shape)
    return ret

# Create output directory
os.makedirs("ablated_outputs_attn1", exist_ok=True)

# Define prompt
prompt = "a banana at the top of the image and an apple at the bottom"

# === Generate and save baseline image ===
print("Generating baseline image (no ablation)")
baseline_image = pipe(
    prompt=prompt,
    height=1024,
    width=1024,
    guidance_scale=5.0,
    num_inference_steps=20,
    generator=torch.Generator(device="cuda").manual_seed(42),
)[0]
baseline_image[0].save("ablated_outputs_attn1/baseline_no_ablation.png")

# === Iterate over transformer blocks and ablate attn1 ===
for idx, block in enumerate(pipe.transformer.transformer_blocks):
    print(f"Mean ablating attn1 in block {idx}")

    # Register hook on attn1
    handle = block.attn1.register_forward_hook(mean_ablation_hook)

    # Generate image with ablated attn1
    image = pipe(
        prompt=prompt,
        height=1024,
        width=1024,
        guidance_scale=5.0,
        num_inference_steps=20,
        generator=torch.Generator(device="cuda").manual_seed(42),  # fresh generator
    )[0]

    # Save image
    image[0].save(f"ablated_outputs_attn1/ablated_block{idx}_attn1_mean.png")

    # Remove hook
    handle.remove()


Generating baseline image (no ablation)


  0%|          | 0/20 [00:00<?, ?it/s]

Mean ablating attn1 in block 0


  0%|          | 0/20 [00:00<?, ?it/s]

torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
Mean ablating attn1 in block 1


...

torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
Mean ablating attn1 in block 18


  0%|          | 0/20 [00:00<?, ?it/s]

torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
Mean ablating attn1 in block 19


  0%|          | 0/20 [00:00<?, ?it/s]

torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
torch.Size([2, 1024, 2240])
